In [ ]:
# Install the transformers library
! pip install transformers

In [ ]:
# Import necessary libraries
import torch
import random

from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Set random seed for reproducibility
random_seed = 42

random.seed(random_seed)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(random_seed)

In [ ]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Load the pre-trained BERT model
model = BertModel.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Print the model architecture
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
# Define two sample texts
text = "hello my name is Saurav Sabu"
text1 = "Hi my name is roshan"

In [ ]:
# Encode the texts using the tokenizer
encoding = tokenizer.batch_encode_plus([text,text1],
                                       padding=True,
                                      truncation=True,
                                      return_tensors="pt",
                                      add_special_tokens=True
                                      )

In [ ]:
# Extract the input IDs (token IDs) from the encoding
input_ids = encoding["input_ids"]

In [ ]:
# Print the input IDs
input_ids

tensor([[  101,  7592,  2026,  2171,  2003,  7842,  4648,  2615,  7842,  8569,
           102],
        [  101,  7632,  2026,  2171,  2003, 20996,  9688,   102,     0,     0,
             0]])

In [ ]:
# Extract the attention mask from the encoding
attention_mask = encoding["attention_mask"]

In [ ]:
# Print the attention mask
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])

In [ ]:
# Perform a forward pass through the model to get the outputs
with torch.no_grad():
  output = model(input_ids,attention_mask = attention_mask)
  # Extract the embeddings for the entire sentences
  sentence_embedding = output.last_hidden_state

In [ ]:
# Print the shape of the sentence embeddings tensor
sentence_embedding.shape

torch.Size([2, 11, 768])

In [ ]:
# Print the sentence embeddings tensor
sentence_embedding

tensor([[[-0.1632,  0.2026, -0.1074,  ..., -0.2942,  0.1224,  0.5295],
         [ 0.1941,  0.4762,  0.3129,  ..., -0.1304,  0.4387,  0.1726],
         [-0.1487, -0.1460, -0.1720,  ..., -0.1117, -0.2135,  0.1597],
         ...,
         [-0.8466, -0.7417, -0.6725,  ..., -0.1558,  0.5695,  0.1713],
         [-0.3761,  0.4187,  0.0203,  ...,  0.4135,  0.1104,  0.5234],
         [ 0.6299,  0.0056, -0.3492,  ...,  0.0740, -0.5592, -0.1545]],

        [[-0.2212,  0.4126, -0.0447,  ..., -0.1741,  0.3816,  0.2484],
         [ 0.5477,  0.6676,  0.5604,  ..., -0.2178,  0.8225, -0.4237],
         [-0.0812, -0.1881, -0.7707,  ...,  0.0820, -0.0511,  0.2533],
         ...,
         [-0.4171,  0.1018,  0.3608,  ...,  0.2714,  0.2315, -0.2180],
         [-0.4039,  0.0796,  0.3139,  ...,  0.3106,  0.2434, -0.1917],
         [-0.4619,  0.0733,  0.3308,  ...,  0.3063,  0.2036, -0.1566]]])

In [ ]:
# Decode the first input ID sequence back to a string, skipping special tokens
tokenizer.decode(input_ids[0],skip_special_tokens=True)

'hello my name is saurav sabu'

In [ ]:
# Decode the second input ID sequence back to a string, skipping special tokens
tokenizer.decode(input_ids[1],skip_special_tokens=True)

'hi my name is roshan'